![logo](./finspace_logo.png)

In [ ]:
%local
from aws.finspace.cluster import FinSpaceClusterManager

# if this was already run, no need to run again
if 'finspace_clusters' not in globals():
    finspace_clusters = FinSpaceClusterManager()
    finspace_clusters.auto_connect()
else:
    print(f'connected to cluster: {finspace_clusters.get_connected_cluster_id()}')

In [ ]:
#####----------------------------------------------------------
##### REPLACE WITH CORRECT IDS!
##### Dataset: "US Equity Time-Bar Summary - 1 min, 14 Symbols"
#####
#####----------------------------------------------------------
dataset_id = ''
view_id    = ''

In [ ]:
# import needed libraries
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import pyspark.sql.functions as F
import pyspark.sql.types as T

from aws.finspace.timeseries.spark.util import string_to_timestamp_micros
from aws.finspace.timeseries.spark.analytics import *
from aws.finspace.timeseries.spark.windows import *

In [ ]:
from aws.finspace.analytics import FinSpaceAnalyticsManager
finspace = FinSpaceAnalyticsManager(spark = spark)

sumDF = finspace.read_data_view(dataset_id = dataset_id, data_view_id = view_id)

In [ ]:
sumDF.show(5)

In [ ]:
# for now, limit the dates involved
sDate = dt.datetime(2019, 10, 1)
eDate = dt.datetime(2019, 10, 1)

df = ( sumDF.filter(sumDF.eventtype == "TRADE NB").filter( sumDF.date.between(sDate, eDate) ) )

In [ ]:
df.show(5)

# Available Time Series Features

In [ ]:
# all functions have help, like this
help(time_series_forecast_indicator)

# All Indicators in one DataFrame
This is an example of how one can combine any or all the indicator functions into very wide DataFrames

In [ ]:
tsDF = df

# group the sets of values
partitionList = ["ticker", "eventtype"]

tenor = 15
numStd = 2

timeCol  = 'end'
priceCol = 'vwap'
highCol  = 'high'
lowCol   = 'low'
volCol   = 'volume'

ts_dict = {
    'moving_average'                       : moving_average( tenor, timeCol, priceCol ),
    'realized_volatility'                  : realized_volatility( tenor, timeCol, priceCol ),
    'exponential_moving_average'           : exponential_moving_average( tenor, timeCol, priceCol ),
    'moving_average_converge_diverge'      : moving_average_converge_diverge( 12, 26, timeCol, priceCol ),
    'moving_average_converge_diverge_hist' : moving_average_converge_diverge_hist( 12, 26, 9, timeCol, priceCol ),
    
    'momentum_indicator'                   : momentum_indicator( tenor, timeCol, priceCol ),
    'roc_indicator'                        : roc_indicator( tenor, timeCol, priceCol ),
    'rsi'                                  : rsi( tenor, timeCol, priceCol ),
    'realized_volatility_spread'           : realized_volatility_spread( tenor, timeCol, priceCol, 'open' ),
    'pairwise_realized_correlation'        : pairwise_realized_correlation( tenor, timeCol, priceCol, 'open' ),
    
    'bollinger_bands'                      : bollinger_bands( tenor, numStd, timeCol, priceCol, highCol, lowCol ),
    'acc_dist_indicator'                   : acc_dist_indicator( timeCol, priceCol, highCol, lowCol, volCol ),
    
    'average_true_range'                   : average_true_range( tenor, timeCol, priceCol, highCol, lowCol ),
    'commodity_channel_index'              : commodity_channel_index( tenor, timeCol, priceCol, highCol, lowCol ),
    'ichimoku_indicator'                   : ichimoku_indicator( timeCol, priceCol, 9, 26, 52 ),
    'linear_weighted_moving_average'       : linear_weighted_moving_average( tenor, timeCol, priceCol ),
    'on_balance_volume'                    : on_balance_volume( timeCol, priceCol, volCol ),
    
    'slow_stock_oscillator'                : slow_stock_oscillator( tenor, timeCol, priceCol, highCol, lowCol ),
    'fast_stock_oscillator'                : fast_stock_oscillator( tenor, timeCol, priceCol, highCol, lowCol ),
    'aroon_up_indicator'                   : aroon_up_indicator( tenor, timeCol, priceCol ),
    'aroon_down_indicator'                 : aroon_down_indicator( tenor, timeCol, priceCol ),
    'aroon_oscillator'                     : aroon_oscillator( tenor, timeCol, priceCol ),
    
    'will_r_indicator'                     : will_r_indicator( tenor, timeCol, priceCol, highCol, lowCol ),
    'cmo_indicator'                        : cmo_indicator( tenor, timeCol, priceCol ),
    'natr_indicator'                       : natr_indicator( tenor, timeCol, priceCol, highCol, lowCol ),
    'pvt_indicator'                        : pvt_indicator( timeCol, priceCol, volCol ),
    'keltner_indicator'                    : keltner_indicator( timeCol, priceCol, highCol, lowCol, 2, 20, 20),
    
    'ult_osc_indicator'                    : ult_osc_indicator( timeCol, priceCol, highCol, lowCol ),
    'apo_indicator'                        : apo_indicator( 12, 26, timeCol, priceCol ),
    'double_exponential_moving_average'    : double_exponential_moving_average( tenor, timeCol, priceCol ),
    'mf_indicator'                         : mf_indicator( tenor, timeCol, priceCol, highCol, lowCol, volCol ),
    'triple_exponential_moving_average'    : triple_exponential_moving_average( tenor, timeCol, priceCol ),
    
    'ts_linear_regression'                 : ts_linear_regression( tenor, 'seconds', timeCol, timeCol, priceCol ),
    'weighted_ts_linear_regression'        : weighted_ts_linear_regression( tenor, 'seconds', timeCol, timeCol, priceCol, volCol ),
    'percentage_price_oscillator'          : percentage_price_oscillator( 12, 26, 9, timeCol, priceCol ),
    'ROC100_indicator'                     : ROC100_indicator( tenor, timeCol, priceCol ),

    'ROCP_indicator'                       : ROCP_indicator( tenor, timeCol, priceCol ),
    'ROCR_indicator'                       : ROCR_indicator( tenor, timeCol, priceCol ),
    'price_channel_indicator'              : price_channel_indicator( tenor, timeCol, priceCol ),
    'trima_indicator'                      : trima_indicator( tenor, timeCol, priceCol ),
    'trix_indicator'                       : trix_indicator( tenor, timeCol, priceCol ),

    'midprice_indicator'                   : midprice_indicator( tenor, timeCol, highCol, lowCol ),
    'midpoint_indicator'                   : midpoint_indicator( tenor, timeCol, priceCol ),
    'min_indicator'                        : min_indicator( tenor, timeCol, priceCol ),
    'max_indicator'                        : max_indicator( tenor, timeCol, priceCol ),
    'minmax_indicator'                     : minmax_indicator( tenor, timeCol, priceCol ),

    'kama_indicator'                       : kama_indicator( timeCol, priceCol, 10, 30, 2 ),
    'adx_indicator'                        : adx_indicator( tenor, timeCol, priceCol, highCol, lowCol ),
    't3_ema_indicator'                     : t3_ema_indicator( tenor, timeCol, priceCol ),
    'wilder_smoothing_indicator'           : wilder_smoothing_indicator( tenor, timeCol, priceCol ),
    'sar_indicator'                        : sar_indicator( timeCol, priceCol, highCol, lowCol ),

    'stddev_indicator'                     : stddev_indicator( tenor, timeCol, priceCol ),
    'dmi_indicator'                        : dmi_indicator( tenor, timeCol, priceCol, highCol, lowCol ),
    'tr_indicator'                         : tr_indicator( timeCol, priceCol, highCol, lowCol ),
    'pos_dm_indicator'                     : pos_dm_indicator( timeCol, highCol, lowCol ),
    'neg_dm_indicator'                     : neg_dm_indicator( timeCol, highCol, lowCol ),

    'chaiken_money_flow_indicator'         : chaiken_money_flow_indicator( tenor, timeCol, priceCol, highCol, lowCol, volCol ),
    'force_index_indicator'                : force_index_indicator( tenor, timeCol, priceCol, volCol ),
    'negative_volume_indicator'            : negative_volume_indicator( 1, timeCol, priceCol, volCol ),
    'donchian_channel_indicator'           : donchian_channel_indicator( tenor, timeCol, highCol, lowCol ),

    'mass_index_indicator'                 : mass_index_indicator( 9, 25, timeCol, highCol, lowCol ),
    'dpo_indicator'                        : dpo_indicator( tenor, timeCol, priceCol ),
    'coppock_curve_indicator'              : coppock_curve_indicator( 14, 11, 10, timeCol, priceCol ),
    'elder_ray_index_indicator'            : elder_ray_index_indicator( tenor, timeCol, priceCol ),
    'fisher_transformation_indicator'      : fisher_transformation_indicator( tenor, timeCol, highCol, lowCol ),

    'chaikens_volatility_indicator'        : chaikens_volatility_indicator( tenor, timeCol, highCol, lowCol ),
    'klinger_oscillator_indicator'         : klinger_oscillator_indicator( 34, 55, timeCol, priceCol, highCol, lowCol, volCol ),
    'hull_moving_average_indicator'        : hull_moving_average_indicator( tenor, timeCol, priceCol ),
    'time_series_forecast_indicator'       : time_series_forecast_indicator( tenor, 'seconds', timeCol, timeCol, priceCol ),
    'stoch_rsi_indicator'                  : stoch_rsi_indicator( tenor, tenor, timeCol, priceCol ),

    'typical_price_indicator'              : typical_price_indicator( tenor, timeCol, priceCol, highCol, lowCol ),
    'weighted_close_indicator'             : weighted_close_indicator( tenor, timeCol, priceCol, highCol, lowCol ),
    'adrx_indicator'                       : adrx_indicator( tenor, tenor, timeCol, priceCol, highCol, lowCol ),

}

for key, fnc in ts_dict.items():
    tsDF = compute_analytics_on_features(tsDF, key, fnc, partition_col_list = partitionList)

tsDF.printSchema()    

In [ ]:
# Compute and Sample some rows 
# 2*tenor: get past the initial NAs that are computed for less-full windows

tsDF.show(2*tenor)

In [ ]:
import datetime
print( f"Last Run: {datetime.datetime.now()}" )